In [1]:
import os
import sys
from langchain_community.vectorstores import Chroma
from IPython.display import display, Markdown

current_dir = os.getcwd()
kit_dir = os.path.abspath(os.path.join(current_dir, ".."))
repo_dir = os.path.abspath(os.path.join(kit_dir, ".."))

sys.path.append(kit_dir)
sys.path.append(repo_dir)

from src.routing import Router
from enterprise_knowledge_retriever.src.document_retrieval import DocumentRetrieval

CONFIG_PATH = os.path.join(kit_dir,'config.yaml')

/Users/feiwu/Documents/ai-starter-kit-personal/ai-starter-kit/routing/routing_env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/feiwu/Documents/ai-starter-kit-personal/ai-starter-kit/routing/routing_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


WANDB_API_KEY is not set. Weave initialization skipped.


# init router

In [2]:
router = Router(CONFIG_PATH)

# init RAG

In [3]:
document_retrieval =  DocumentRetrieval()
embeddings = document_retrieval.load_embedding_model()
# print(os.path.join(kit_dir, "data/800_20"))
vectorstore = document_retrieval.load_vdb(db_path = os.path.join(kit_dir, "data/800_20"), embeddings=embeddings)
document_retrieval.init_retriever(vectorstore)
conversation = document_retrieval.get_qa_retrieval_chain()

2024-09-19 17:17:48.296 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2024-09-19 17:18:02,296 [INFO] - Load pretrained SentenceTransformer: intfloat/e5-large-v2


load INSTRUCTOR_Transformer


2024-09-19 17:18:07,977 [INFO] - Use pytorch device: cpu


max_seq_length  512
Loading collection name is None


2024-09-19 17:18:10,188 [INFO] - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


# user input

In [4]:
query = "How to measure performance using sambatune?"
# query = "Who is the CEO of Sambanova?"
# query = "What is LLM?"

# pipeline

In [5]:
datasource = router.routing(query)
if datasource == "vectorstore":
    output = conversation.invoke({"question":query})
    response = output['answer']
else:
    response = document_retrieval.llm(query)

print(response)
print("--------------------------")
print(f"datasource: {datasource}")

/Users/feiwu/Documents/ai-starter-kit-personal/ai-starter-kit/routing/routing_env/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


LLM stands for Large Language Model. It's a type of artificial intelligence (AI) model that is trained on a massive corpus of text data to generate human-like language. LLMs are designed to understand and generate text, and they can be used for a wide range of applications, such as:

1. **Language translation**: LLMs can translate text from one language to another.
2. **Text summarization**: LLMs can summarize long pieces of text into shorter, more digestible versions.
3. **Chatbots**: LLMs can be used to power chatbots that can have conversations with humans.
4. **Content generation**: LLMs can generate text, such as articles, stories, and even entire books.
5. **Question answering**: LLMs can answer questions based on the text they've been trained on.

LLMs work by using a technique called **deep learning**, which involves training a neural network on a large dataset of text. The neural network learns to recognize patterns in the text and generate new text based on those patterns.

S